In [1]:
import fluids
from fluids.units import *

Given:  600 psig steam at 850 degF flows through 400 feet of horizontal 6" Schedule 80 pipe at a rate of 90,000 pounds per hour.  The system contains three 90 degree weld elbows having a relative radius of 1.5, one fully-open 6x4" Class 600 venturi gate valve with inlet and outlet ports conically tapered from back of body righs to valve ends.  Face-to-face dimensions is 22" and back of seat ring is about 6".   Additionally, there is one 6" Class 600 y-pattern globe valve.  Latter has a seat diameter equal to 0.9 of the inside diameter of Schedule 80 pipe, disc fully lifted.
Find:  The pressure drop through the system

K2, L/D and L should be given in terms of 6" Schedule 80 pipe

In [2]:
V_bar = ((1.3023+1.1904)/2)*u.feet**3/u.lb  #specific volume, Crane 410, A-18
mu = 0.027*u.cP #Crane 410, A-2

ft_pipe = 0.015 #Crane 410, A-27

V_bar, mu, ft_pipe

(<Quantity(1.24635, 'foot ** 3 / pound')>,
 <Quantity(0.027, 'centipoise')>,
 0.015)

# Solve for K

In [3]:
#K for Venturi Gate Valve
d1 = (4.0*u.inch).to(u.meter)  #valve seat bore is 4.0"
d2 = nearest_pipe(NPS=6*u.inch, schedule='80')[1]

beta = d1/d2

ft = 0.015 #Crane 410, A-27
K1 = 8*ft #from Crane 410 A-28

tan_theta_div2 = 0.5*(d2.to(u.inch)-d1.to(u.inch))/(0.5*(22*u.inch-6*u.inch))
sin_theta_div2 = tan_theta_div2 #approximately accurate for small theta


K2 = (K1+sin_theta_div2*(0.8*(1-beta**2)+2.6*(1-beta**2)**2))/beta**4 #from Crane 410 A-27

K_venturi = K2

LD = K2/ft

L = (LD*d2.to(u.inch)).to(u.feet)

print("K venturi is: {}, L/D is: {}, L is {}".format(K_venturi, LD, L))



K venturi is: 1.0443998252277853 dimensionless, L/D is: 69.62665501518569 dimensionless, L is 33.433586706110816 foot


In [4]:
#K for globe valve
ft = 0.015 #Crane 410, A-27, for 6"
beta = 0.9
K1 = 55*ft
K2 = K1 + beta*(0.5*(1-beta**2)+(1-beta**2)**2)/(beta**4)
K_globe = K2
K_globe

1.004835390946502

In [5]:
#K for elbows
ft = 0.015 #Crane 410, A-27, for 6"
K = 14*ft
K_elbow = 3*K

K_elbow

0.63

In [6]:
#K for pipe
ft = 0.015 #Crane 410, A-27, for 6"
L = 400*u.feet
D = d2.to(u.feet)
K = ft*L/D

K_pipe = K

In [7]:
Ktot = K_pipe + K_elbow + K_globe + K_venturi
Ktot

<Quantity(15.174452488653104, 'dimensionless')>

# Re

In [8]:
W = 90000*u.lb/u.hour #steam flow rate
Re = (6.315*W/(d2.to(u.inch)*mu)).magnitude

Re

3653115.6053566546

Fully turbulent flow at this Re

# delta P

In [9]:
delta_P = ((2.799e-7)*Ktot*(W**2)*V_bar/(d2.to(u.inch)**4)).magnitude*u.psi

delta_P.round(1)

<Quantity(38.9, 'pound_force_per_square_inch')>